In [42]:
import numpy as np
import pandas as pd
import typing as tp

Общие ошибки:
1. Использование циклов и list comprehension, map и Collections там, где это не нужно (а это нигде не нужно)
2. Ненужные apply (везде, кроме dead_lucky, можно найти встроенные функции)
3. Ненужные присваивания
4. Модификация датафрейма

#### numpy basic

In [116]:
def construct_array(
        matrix: tp.Any,
        row_indices: tp.List[int],
        col_indices: tp.List[int]) -> tp.Any:
    return matrix[np.array(row_indices), np.array(col_indices)]

def mean_channel(X: tp.Any) -> tp.Any:
    return np.mean(np.array(X), axis=(1, 0))

def get_unique_rows(X: tp.Any) -> tp.Any:
    return np.unique(np.array(X), axis=0)

def detect_identic(
        lhs_array: tp.Any,
        rhs_array: tp.Any) -> bool:
    return np.array_equal(lhs_array, rhs_array)

def construct_matrix(first_array: tp.Any, second_array: tp.Any) -> tp.Any:
    return np.column_stack((first_array, second_array))

#### nearest_value

In [91]:
def nearest_value(matrix: tp.Any, value: float) -> tp.Optional[float]:
    result = np.argmin(np.abs(matrix - value))
    return matrix.ravel()[result]


**flatten** -  создает копию, **ravel** - нет

#### vander

In [100]:
def vander(array: tp.Any) -> tp.Any:
    degrees = np.arange(array.shape[0])
    return np.power(array[:, np.newaxis], degrees)

Так нормально:

In [97]:
def vander(array: tp.Any) -> tp.Any:
    n = array.shape[0]
    res = np.logspace(np.zeros_like(array), (n-1), base=array, num=n)
    return res.T

Так плохо:

In [95]:
def vander(array: tp.Any) -> tp.Any:
    return np.array([[pow(elem, i) for i in range(array.shape[0])] for elem in array])

#### add_zeros

In [106]:
def add_zeros(x: tp.Any) -> tp.Any:
    return np.insert(x, range(1, len(x)), 0)

### Titanic

In [127]:
df = pd.read_csv("titanic.csv", sep="\t")

#### male_age

In [4]:
def male_age(df):
    return df[(df["Survived"] == 1) &
              (df["Sex"] == "male") &
              (df["Embarked"] == "S") &
              (df["Fare"] > 30)]["Age"].mean()

#### nan_columns

In [ ]:
def nan_columns(df):
    return df.columns[df.isna().any()]

Так плохо:

In [ ]:
def nan_columns(df):
    return set(df.columns) - set(df.dropna(axis=1).columns)

In [ ]:
def nan_columns(df):
    return list(df.apply(lambda col: col.isnull().value_counts()).T[True].dropna().index)

Возмутительно:

In [6]:
def nan_columns(df):
    return [x for x in df.columns if np.any(pd.isna(df[x]).values)]

#### class_distribution

value_counts -> для одной колонки

groupby + size -> для нескольких

In [7]:
def class_distribution(df):
    return df['Pclass'].value_counts(normalize=True).sort_index()

Нормально:

In [ ]:
def class_distribution(df: tp.Any) -> tp.Any:
    return df['Pclass'].value_counts(ascending=True) / df['Pclass'].count()

Плохо:

In [ ]:
def class_distribution(df: tp.Any) -> tp.Any:
    A = df["Pclass"].value_counts() / 156
    return A.sort_index()

#### families_count

split можно делать методами pandas

Хорошо:

In [134]:
def families_count(df, k):
    members = df["Name"].str.split(",").str[0].value_counts()
    return members[members > k].shape[0]

In [130]:
def families_count(df: tp.Any, k: int) -> float:
    return (df.groupby(df["Name"].str.split(',', 1).str[0])['Name'].count() > k).sum()

Плохо:

In [ ]:
def families_count(df, k):
    df.Name = df.Name.apply(lambda x: x.split(',')[0])
    return len([i for i in df.groupby(df.Name).size().values if i > k])

#### max_size_group

In [138]:
def max_size_group(df: tp.Any, columns: tp.Any) -> tp.Any:
    return df.groupby(columns).size().idxmax()

In [145]:
def max_size_group(df, columns):
    return df.groupby(columns)['PassengerId'].count().idxmax()

#### lucky_dead

Нормально:

In [159]:
def sum_of_digits(number: str) -> int:
    return sum(map(int, number))
               
def is_lucky(ticket: str) -> bool:
    if ticket.isdecimal() and (len(ticket) % 2 == 0) and \
            sum_of_digits(ticket[:len(ticket) // 2]) == \
            sum_of_digits(ticket[len(ticket) // 2:]):
        return True
    return False

def dead_lucky(df: tp.Any) -> float:
    lucky = df[df["Ticket"].apply(is_lucky)]["Survived"]
    return lucky.value_counts(normalize=True)[0]

Плохо:

In [160]:
def dead_lucky(df: tp.Any) -> float:
    def is_lucky(t: tp.Any) -> int:
        try:
            t = int(t)
        except Exception:
            return 0
        t = str(t)
        if len(t) % 2 != 0:
            return 0
        ans = 0
        for i, c in enumerate(t):
            if i < len(t) / 2:
                ans += int(c)
            else:
                ans -= int(c)
        return int(ans == 0)
    df['Ticket'] = df['Ticket'].apply(is_lucky)
    df = df[df['Ticket'] == 1]
    df = df.groupby('Survived').size()
    df = df / df.sum()
    return df[0]

Не нужно делать копию и модифицировать датафрейм без необходимости:

In [ ]:
def dead_lucky(df: tp.Any) -> float:
    df["is_lucky"] = df["Ticket"].apply(is_lucky)
    return 1 - df[df["is_lucky"]]["Survived"].mean()

In [ ]:
def dead_lucky(df: tp.Any) -> float:
    df_new = df.copy()
    df_new["is_lucky"] = df["Ticket"].apply(is_lucky)
    return 1 - df_new[df_new["is_lucky"]]["Survived"].mean()

Лучше создать локальную переменную

In [ ]:
def dead_lucky(df: tp.Any) -> float:
    is_lucky = df["Ticket"].apply(is_lucky)
    return 1 - df[is_lucky]["Survived"].mean()

Если датафрейм маленький, лучше в начале сделать копию (df.copy()) и спокойно работать, не изменяя исходного датафрейма

In [161]:
df = pd.DataFrame([[1,2,3], [4,5,6]], columns=["A", "B", "C"])

def func_copy(df):
    df = df.copy()
    df["A"] = 123

func_copy(df)
display(df)

def func(df):
    df = pd.DataFrame(df)
    df["A"] = 123

func(df)
display(df)

,A,B,C
0,1,2,3
1,4,5,6


,A,B,C
0,123,2,3
1,123,5,6
